In [1]:
import pandas as pd
#import MetaTrader5 as mt5
import talib as ta
from bb_setup_1 import setup
import warnings
warnings.filterwarnings("ignore")

In [3]:
ativo = 'win_1_full'
colnames = ['Data','Open','High','Low','Close','Vol']
df = pd.read_csv(f'./Dados/{ativo}.csv', names = colnames, skiprows=1)
df = df[df['Data'] > '2022-06-01']

In [43]:
ativo[:-5]

'win_1'

In [44]:
pontos = 20
rate_stop = 1
rate_tp = 2
spread = 5

df_total = None

for timeframe in [1,5,15]:
    for var_bb in [2,2.5,3]:
        for pontos in [20,40,80,100]:
            for rate_tp in [1.5,2,2.5,3,5]:
                
                ativo = f'win_{timeframe}_full'
                colnames = ['Data','Open','High','Low','Close','Vol']
                df = pd.read_csv(f'./Dados/{ativo}.csv', names = colnames, skiprows=1)
                df = df[df['Data'] > '2022-06-01']

                df_acao = setup(df,var_bb = var_bb,time_period = 20,pontos = pontos,rate_stop = 1,rate_tp = rate_tp)

                qtd_operacoes = df_acao.shape[0]
                taxa_acerto = df_acao['resultado_binario'].sum()/qtd_operacoes
                operacoes_certas = qtd_operacoes * taxa_acerto
                operacoes_erradas = qtd_operacoes - operacoes_certas
                pontos_bruto =  (operacoes_certas * rate_tp - operacoes_erradas * rate_stop) * pontos

                pontos_liquido = pontos_bruto - spread * qtd_operacoes


                df_new = pd.DataFrame([[ativo[:-5],rate_tp,rate_stop,pontos,qtd_operacoes,taxa_acerto,pontos_bruto,pontos_liquido]],
                            columns = ['ativo','rate_tp','rate_stop','pontos','qtd_operacoes','taxa_acerto','pontos_bruto','pontos_liquido'])

                df_total = pd.concat([df_total,df_new])

                print(df_total.to_string())

   ativo  rate_tp  rate_stop  pontos  qtd_operacoes  taxa_acerto  pontos_bruto  pontos_liquido
0  win_1      1.5          1      20           1690     0.749704       29550.0         21100.0
   ativo  rate_tp  rate_stop  pontos  qtd_operacoes  taxa_acerto  pontos_bruto  pontos_liquido
0  win_1      1.5          1      20           1690     0.749704       29550.0         21100.0
0  win_1      2.0          1      20           1690     0.633728       30460.0         22010.0
   ativo  rate_tp  rate_stop  pontos  qtd_operacoes  taxa_acerto  pontos_bruto  pontos_liquido
0  win_1      1.5          1      20           1690     0.749704       29550.0         21100.0
0  win_1      2.0          1      20           1690     0.633728       30460.0         22010.0
0  win_1      2.5          1      20           1690     0.538462       29900.0         21450.0
   ativo  rate_tp  rate_stop  pontos  qtd_operacoes  taxa_acerto  pontos_bruto  pontos_liquido
0  win_1      1.5          1      20           169

In [47]:
df_1 = df_total[df_total['ativo'] == 'win_1']
df_5 = df_total[df_total['ativo'] == 'win_5']
df_15 = df_total[df_total['ativo'] == 'win_15']

In [50]:
df_1.sort_values(by = 'pontos_liquido', ascending = False).head(10)

,ativo,rate_tp,rate_stop,pontos,qtd_operacoes,taxa_acerto,pontos_bruto,pontos_liquido
0,win_1,2.0,1,20,1690,0.633728,30460.0,22010.0
0,win_1,2.5,1,20,1690,0.538462,29900.0,21450.0
0,win_1,1.5,1,20,1690,0.749704,29550.0,21100.0
0,win_1,3.0,1,20,1690,0.456213,27880.0,19430.0
0,win_1,1.5,1,40,1690,0.533136,22500.0,14050.0
0,win_1,3.0,1,20,734,0.539510,17000.0,13330.0
0,win_1,2.5,1,20,734,0.615804,16960.0,13290.0
0,win_1,2.0,1,20,734,0.693460,15860.0,12190.0
0,win_1,1.5,1,20,734,0.803815,14820.0,11150.0
0,win_1,2.0,1,40,1690,0.426036,18800.0,10350.0


In [ ]:
df_1.sort_values(by = 'pontos_liquido', ascending = False).head(10)

In [153]:
var_bb = 2.5
time_period = 20
df['BB_up'],df['BB_mid'],df['BB_low'] = ta.BBANDS(df['Close'], timeperiod=time_period, nbdevup=var_bb, nbdevdn=var_bb, matype=0)

df['tamanho_corpo'] = abs(df['Open'] - df['Close'])
df['tamanho_pra_baixo'] = abs(df['Open'] - df['Low'])
df['tamanho_abaixo'] = abs(df['Close'] - df['Low'])
df['tamanho_pra_cima'] = abs(df['Open'] - df['High'])
df['tamanho_acima'] = abs(df['High'] - df['Close'])
df['proximo_candle'] = df['Close'].shift(-1) - df['Open'].shift(-1)

df['pos_BB_inicial'] = df.apply(lambda x: 1 if x['Close'] > x['BB_up'] else -1 if x['Close'] < x['BB_low'] else 0, axis = 1)
#df['pos_BB'] = df['pos_BB_inicial'].diff()

#pos_BB = -2 cruzou pra BB_low pra baixo // pos_BB = 2 cruzou BB_high pra cima // 

df['verifica_tamanho'] = df.apply(lambda x: x['tamanho_corpo']/(x['tamanho_pra_baixo'] + 1e-4) if x['pos_BB_inicial'] == -1 else
                                     x['tamanho_corpo']/(x['tamanho_pra_cima'] + 1e-4) if x['pos_BB_inicial'] == 1 else 0, axis = 1)

percent_fechamento = 0.9

df['acao'] = df.apply(lambda x: 'call' if (x['pos_BB_inicial'] == -1 and x['verifica_tamanho'] >= percent_fechamento) else
                                 'sell' if (x['pos_BB_inicial'] == 1 and x['verifica_tamanho'] >= percent_fechamento) else 0 , axis = 1) 


lista = df['acao'].values
tempo_lista =10
for num, i in enumerate(lista):
    #print(num,i)
    if i != 0:
        for i in range(tempo_lista):
            try:
                n_num = num + i + 1
                lista[n_num] = 0
            except:
                pass

df['acao'] = lista    

'''df['resultado_binario'] = df.apply(lambda x: 1 if (x['acao'] == 'call' and x['proximo_candle'] > 0) else 1 
                                                if (x['acao'] == 'sell' and x['proximo_candle'] < 0) else 0, axis = 1)'''

pontos = 20
#rate_stop é somado (multiplicador por tamanho de candle) em uma unidade de tamanho. rate 0.5 = stop 1.5 (somar 1)
rate_stop = 1
df['stop'] = df.apply(lambda x: x['Close'] + (pontos * rate_stop)  if x['acao'] == 'sell' else 
x['Close'] - (pontos * rate_stop) if x['acao'] == 'call' else 0, axis = 1)

#rate_tp é somado em 0 unidades de tamanho. rate_tp 2 = tp 2  --Valor igual rate
rate_tp = 1.5
df['tp'] = df.apply(lambda x: x['Close'] - rate_tp * pontos if x['acao'] == 'sell' else
                                        x['Close'] + rate_tp * pontos if x['acao'] == 'call' else 0, axis =1)

for index, row in df.iterrows():
    if row['acao'] == 'call':
        i = 1
        while True:
            try:
                if df.loc[index + i,'High'] >= row['tp']:
                    df.loc[index,'resultado_binario'] = 1
                    break
                elif df.loc[index + i,'Low'] <= row['stop']:
                    df.loc[index,'resultado_binario'] = 0
                    break
                i+=1
            except:
                df.loc[index,'resultado_binario'] = 0
                break
    elif row['acao'] == 'sell':
        i = 1
        while True:
            try:
                if df.loc[index + i,'Low'] <= row['tp']:
                    df.loc[index,'resultado_binario'] = 1
                    break
                elif df.loc[index + i,'High'] >= row['stop']:
                    df.loc[index,'resultado_binario'] = 0
                    break
                i+=1
            except:
                df.loc[index,'resultado_binario'] = 0
                break


In [154]:
df_acao = df[(df['acao'] != 0) & ((df['verifica_tamanho'] <=1))]
df_acao['resultado_binario'].sum()/df_acao.shape[0]

0.8283018867924529

In [155]:
df_acao[['Data',
       'verifica_tamanho', 'acao','resultado_binario']].tail(30)

,Data,verifica_tamanho,acao,resultado_binario
96668,2022-12-15 10:00:00,0.942857,call,1.0
96750,2022-12-15 11:22:00,0.965517,sell,1.0
96788,2022-12-15 12:00:00,0.905660,call,1.0
96811,2022-12-15 12:23:00,0.975609,call,0.0
96923,2022-12-15 14:15:00,0.999999,sell,1.0
97010,2022-12-15 15:42:00,0.960784,call,1.0
97126,2022-12-15 17:38:00,0.999999,call,1.0
97172,2022-12-15 18:24:00,0.999999,sell,1.0
97306,2022-12-16 11:13:00,0.999999,call,0.0
97668,2022-12-16 17:15:00,0.925000,sell,1.0


In [27]:
df.loc[99955:][['Data','BB_up', 'BB_mid',
       'BB_low', 'tamanho_corpo', 'tamanho_pra_baixo', 'tamanho_abaixo',
       'tamanho_pra_cima', 'tamanho_acima', 'pos_BB_inicial', 'pos_BB',
       'verifica_tamanho', 'acao']]

,Data,BB_up,BB_mid,BB_low,tamanho_corpo,tamanho_pra_baixo,tamanho_abaixo,tamanho_pra_cima,tamanho_acima,pos_BB_inicial,pos_BB,verifica_tamanho,acao
99955,2022-12-22 14:50:00,109559.425216,109283.25,109007.074784,85.0,245.0,160.0,15.0,100.0,-1,0.0,0.000000,0
99956,2022-12-22 14:55:00,109574.931604,109246.00,108917.068396,160.0,210.0,50.0,40.0,200.0,-1,0.0,0.000000,0
99957,2022-12-22 15:00:00,109680.246403,109193.75,108707.253597,390.0,420.0,30.0,15.0,405.0,-1,0.0,0.000000,0
99958,2022-12-22 15:05:00,109703.352490,109153.25,108603.147510,175.0,85.0,260.0,190.0,15.0,-1,0.0,0.000000,0
99959,2022-12-22 15:10:00,109686.487562,109131.25,108576.012438,325.0,0.0,325.0,355.0,30.0,0,1.0,0.915493,sell
99960,2022-12-22 15:15:00,109675.553298,109115.00,108554.446702,20.0,45.0,65.0,90.0,70.0,0,0.0,0.000000,0
99961,2022-12-22 15:20:00,109661.478508,109084.25,108507.021492,200.0,210.0,10.0,5.0,205.0,0,0.0,0.000000,0
99962,2022-12-22 15:25:00,109661.773151,109046.25,108430.726849,180.0,250.0,70.0,25.0,205.0,0,0.0,0.000000,0
99963,2022-12-22 15:30:00,109656.570880,109000.50,108344.429120,110.0,180.0,70.0,25.0,135.0,0,0.0,0.000000,0
99964,2022-12-22 15:35:00,109624.141858,108957.25,108290.358142,80.0,235.0,315.0,95.0,15.0,0,0.0,0.000000,0


In [23]:
df

,Data,Open,High,Low,Close,Vol,BB_up,BB_mid,BB_low,tamanho_corpo,tamanho_pra_baixo,tamanho_abaixo,tamanho_pra_cima,tamanho_acima,pos_BB_inicial,pos_BB,verifica_tamanho,acao
89099,2022-08-01 09:00:00,103825.0,103830.0,103640.0,103675.0,123612,NaN,NaN,NaN,150.0,185.0,35.0,5.0,155.0,0,NaN,0.0,0
89100,2022-08-01 09:05:00,103675.0,103715.0,103585.0,103690.0,93442,NaN,NaN,NaN,15.0,90.0,105.0,40.0,25.0,0,0.0,0.0,0
89101,2022-08-01 09:10:00,103685.0,103695.0,103565.0,103600.0,80489,NaN,NaN,NaN,85.0,120.0,35.0,10.0,95.0,0,0.0,0.0,0
89102,2022-08-01 09:15:00,103605.0,103630.0,103560.0,103585.0,55816,NaN,NaN,NaN,20.0,45.0,25.0,25.0,45.0,0,0.0,0.0,0
89103,2022-08-01 09:20:00,103580.0,103695.0,103560.0,103695.0,84974,NaN,NaN,NaN,115.0,20.0,135.0,115.0,0.0,0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99994,2022-12-22 18:05:00,109335.0,109385.0,109320.0,109380.0,24762,109519.619529,109027.00,108534.380471,45.0,15.0,60.0,50.0,5.0,0,0.0,0.0,0
99995,2022-12-22 18:10:00,109380.0,109435.0,109115.0,109175.0,79736,109520.800597,109052.00,108583.199403,205.0,265.0,60.0,55.0,260.0,0,0.0,0.0,0
99996,2022-12-22 18:15:00,109175.0,109640.0,109170.0,109480.0,76462,109549.279281,109096.75,108644.220719,305.0,5.0,310.0,465.0,160.0,0,0.0,0.0,0
99997,2022-12-22 18:20:00,109485.0,109560.0,109375.0,109410.0,20567,109569.934187,109130.75,108691.565813,75.0,110.0,35.0,75.0,150.0,0,0.0,0.0,0
